## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Loading Data

In [2]:
project_dir = r"/Users/karida/Desktop/AWS_SageMaker_Project"
data_dir = "Data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_dir, data_dir, file_name)
    return pd.read_csv(file_path)

In [4]:
flights = get_data("flight_price")

In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset has missing values in coloumns "Route" and "Total_Stops".
- The data type of some features is not appropriate

# Preliminary Analysis

### 3.2 Checks the Data Types

In [ ]:
flights.head()

In [ ]:
flights.dtypes

In [ ]:
value = flights.Date_of_Journey.iloc[6]

In [ ]:
value

In [ ]:
value = flights.Dep_Time.iloc[6]

In [ ]:
value

### 3.2 Check of Duplicates

In [ ]:
flights.duplicated().sum()

In [ ]:
(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(["Airline", "Date_of_Journey", "Source", "Destination"])
)

### 3.3 Observations

- `Date_of_Journey`, `Dept_Time` and `Arrival_Time` should be of date time data type.
- The types of `Duration` and `Total_Stops` are mixed. It should be of numeric type.
- There are 220 duplicate rows that need to be removed.

# Detailed Analysis

## Airlines

In [ ]:
flights.Airline.unique()

- Some of the values in the Airline columns have inaccurat values that need to be corrected.

In [ ]:
(
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.title() 
)

### Date of Journey

In [ ]:
flights.Date_of_Journey

In [ ]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

## Source

In [ ]:
flights.Source.unique()

## Destination

In [ ]:
flights.Destination.unique()

## Dep_Time

In [ ]:
flights.Dep_Time

In [ ]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

In [ ]:
pd.to_datetime(flights.Dep_Time).dt.time

## Arrival Time

In [ ]:
flights.Arrival_Time

In [ ]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
    .unique()
)

## Duration
- Convert to minutes

In [ ]:
flights.Duration

In [ ]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    #.unique()
)

In [ ]:
flights.iloc[[6474]]

- It is found that the 6474 row has not valid value so better to remove it.

In [ ]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour=lambda df_: (
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minute=lambda df_: (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
)

In [ ]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour=lambda df_: (
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minute=lambda df_: (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=[6474]))
)

In [8]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ", expand=True)
        .set_axis(["hour", "minute"], axis=1)
        .assign(
            hour=lambda df_: (
                df_
                .hour
                .str.replace("h", "")
                .astype(int)
                .mul(60)
            ),
            minute=lambda df_: (
                df_
                .minute
                .str.replace("m", "")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis=1)
    )
    

## Total Stops

In [ ]:
flights.Total_Stops.unique()

In [ ]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True) # It will either search for " stop" or " stops"
    .pipe(lambda ser: pd.to_numeric(ser))
    .unique()
)

## additional info column

In [ ]:
flights.Additional_Info.unique()

# Cleaning Operations

In [9]:
def clean_data(df):
    return(
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include="O").columns 
        })
        .rename(columns = str.lower)
        .assign(
            airline=lambda df_: (
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_journey=lambda df_:pd.to_datetime(df_.date_of_journey, dayfirst=True),
            dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_: (
                df_
                .total_stops
                .replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True) # It will either search for " stop" or " stops"
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info=lambda df_: df_.additional_info.replace("No info", "No_Info")
        )
        .drop(columns='route')

    )

In [10]:
flights_clean_data = clean_data(flights)
flights_clean_data

/var/folders/mb/tryjj4mx7m35b7yd9bx74w3m0000gn/T/ipykernel_12589/3553329703.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
/var/folders/mb/tryjj4mx7m35b7yd9bx74w3m0000gn/T/ipykernel_12589/3553329703.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No_Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No_Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No_Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No_Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No_Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No_Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No_Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No_Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No_Info,12648


# Split Data

In [11]:
flights_clean_split = flights_clean_data.sample(1000)

In [13]:
X = flights_clean_data.drop(columns = "price")
y = flights_clean_data.price.copy()

In [52]:
X_data, X_test, y_data, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.2, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6695, 9) (6695,)
(1674, 9) (1674,)
(2093, 9) (2093,)


## 7. Export the Subsets

In [54]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_dir, data_dir, file_name)

    X.join(y).to_csv(file_path, index=False)
    return pd.read_csv(file_path).head()

In [55]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No_Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No_Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087


In [56]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No_Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No_Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No_Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No_Info,16704


In [57]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No_Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No_Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No_Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No_Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No_Info,4804
